**SPARQL via Python**

In [1]:
pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.7 MB/s eta 0:00:00


**SPARQLWrapper: a python wrapper around a SPARQL service.**


– Installation: https://pypi.org/project/SPARQLWrapper/


– Documentation and examples: https://github.com/RDFLib/sparqlwrapper/

In [2]:
pip install SPARQLWrapper

**Owlready: a package for ontology-oriented programming in Python.**


– Installation: https://pypi.org/project/Owlready2/

 – Documentation: https://owlready2.readthedocs.io/en/latest/intro.html

In [3]:
pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 70.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.44-cp310-cp310-linux_x86_64.whl size=24078644 sha256=1fe24346f2e1f8436f900dc84cb5d157f651ed9a93f9e4765e537189f56a9bb6
  Stored in directory: /root/.cache/pip/wheels/88/e1/04/583b0743b2907f091204baaa0aef9740f5ba5f3d2f6a5aa00d
Successfully built owlready2


**Connection to Google Drive( Google Colab)**

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


**Querying RDF-based Knowledge Graphs via SPARQL**

Use the [playground.ttl](https://drive.google.com/file/d/1phOThrcnyi4fKn5m0TUo0zMDk3jdjBE5/view?usp=sharing) dataset, and the method queryLocalRDFGraph() from the Sample Code below as examples.

**Sample Code: Query A Local Graph**

In [5]:
from rdflib import Graph

def queryLocalGraph():

    #Example from  https://www.stardog.com/tutorials/data-model/

    g = Graph()
    g.parse("/content/drive/MyDrive/playground.ttl", format="ttl")



    print("Loaded '" + str(len(g)) + "' triples.")

    #for s, p, o in g:
    #    print((s.n3(), p.n3(), o.n3()))


    print("Females:")

    qres = g.query(
    """SELECT ?thing ?name where {
      ?thing tto:sex "female" .
      ?thing dbp:name ?name .
    }""")

    for row in qres:
        #Row is a list of matched RDF terms: URIs, literals or blank nodes
        print("%s is female with name '%s'" % (str(row.thing),str(row.name)))


queryLocalGraph()

Loaded '73' triples.
Females:
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'


**Task 1**

Load the [beatles.ttl](https://drive.google.com/file/d/1zOYYo9hy2A2FVyROW2D5Icqg7qtrJWeV/view?usp=sharing) dataset

1.   Query the local graph to find:
      1. the name and songs of all the solo artists
      2. the writer of the song *love_Me_Do*
      3. Print the total triples

      



In [6]:
# Task 1
# Explain each step you perform

# Query to get the name and songs of all the solo artists
from rdflib import Graph, Namespace

# Define the namespace prefix and URI to match the RDF data
ex = Namespace("http://stardog.com/tutorial/")

def loadBeatlesDataset():
    g = Graph()
    g.parse("/content/drive/MyDrive/beatles.ttl", format="ttl")
    print("Loaded '" + str(len(g)) + "' triples.")
    # qres = g.query(
    # """SELECT *where {
    #   ?s ?p ?o.
    # }""")

    # for row in qres:
    #     #Row is a list of matched RDF terms: URIs, literals or blank nodes
    #     print(row)
    return g

def querySoloArtistsAndSongs(graph):
    print("\n\nSolo Artists and Their Songs:")
    qres = graph.query(
    """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    select ?song ?soloArtist where{
    ?soloArtist rdf:type <http://stardog.com/tutorial/SoloArtist>   .
    optional{?song <http://stardog.com/tutorial/writer> ?soloArtist}
}"""
    )

    for row in qres:
        print("\nMember but soloArtist: %s, Song: %s" % (str(row.soloArtist), str(row.song)))


def queryWriterOfLoveMeDo(graph):
    print("\nWriter of the song 'Love Me Do':")

    qres = graph.query(
        """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            select ?writer where{
            <http://stardog.com/tutorial/Love_Me_Do> <http://stardog.com/tutorial/writer> ?writer.
            }"""
    )

    for row in qres:
        print("\n\nWriter: %s" % str(row.writer))

def printTotalTriples(graph):
    print("\nTotal Triples in the Graph: %d" % len(graph))

# Load the Beatles dataset
beatles_graph = loadBeatlesDataset()

# Query for solo artists and their songs
querySoloArtistsAndSongs(beatles_graph)

# Query for the writer of the song "Love Me Do"
queryWriterOfLoveMeDo(beatles_graph)

# Print the total number of triples in the graph
printTotalTriples(beatles_graph)


Loaded '20' triples.


Solo Artists and Their Songs:

Member but soloArtist: http://stardog.com/tutorial/John_Lennon, Song: http://stardog.com/tutorial/Love_Me_Do

Member but soloArtist: http://stardog.com/tutorial/Paul_McCartney, Song: http://stardog.com/tutorial/Love_Me_Do

Member but soloArtist: http://stardog.com/tutorial/Ringo_Starr, Song: None

Member but soloArtist: http://stardog.com/tutorial/George_Harrison, Song: None

Writer of the song 'Love Me Do':


Writer: http://stardog.com/tutorial/John_Lennon


Writer: http://stardog.com/tutorial/Paul_McCartney

Total Triples in the Graph: 20


**Sample Code: Query A remote Graph**

In [26]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pprint import pprint
import time



def queryRemoteGraph(endpoint_url, query, attempts=3):


    try:

        sparql_web = SPARQLWrapper(endpoint_url)
        # Default is XML:
        # https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.Wrapper.html
        sparql_web.setReturnFormat(JSON)
        # sparql_web.setReturnFormat(XML)

        sparql_web.setQuery(query)
        print("Im here now")
        results = sparql_web.query().convert()
        ret = sparql_web.queryAndConvert()
        print("Im here now2")
        for r in ret["results"]["bindings"]:
         print(r)
         print("Im here now3")

      #  Prints JSON file
        print("Im here now to check")
        print("RAW RESULTS IN JSON FORMAT:")
        print(results)


        print("\nRESULTS:")
        print("Im here now4")
        for result in results["results"]["bindings"]:

            #Prints individual results
            print(result["name_laur"]["value"])
            print("Im here now5")




    except:

        print("Query '%s' failed. Attempts: %s" % (query, str(attempts)))
        #time.sleep(60) #to avoid limit of calls, sleep 60s
        attempts-=1
        if attempts>0:
            return queryRemoteGraph(endpoint_url, query, attempts)
        else:
            return None



#See more examples here: https://www.nobelprize.org/about/linked-data-examples/
nobelprize_endpoint = "http://data.nobelprize.org/sparql"
nobelprize_query = """
    SELECT DISTINCT ?name_laur WHERE {
    ?laur <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://data.nobelprize.org/terms/Laureate> .
    ?laur <http://www.w3.org/2000/01/rdf-schema#label> ?name_laur .
    ?laur <http://xmlns.com/foaf/0.1/gender> "female" . }
    """
#Same query with prefixes
nobelprize_query = """
    PREFIX nobel: <http://data.nobelprize.org/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?name_laur WHERE {
    ?laur rdf:type nobel:Laureate .
    ?laur rdfs:label ?name_laur .
    ?laur foaf:gender "female" . }
    """



print("\nQuerying Nobel Prize Knowledge Graph (Female laureates):")
queryRemoteGraph(nobelprize_endpoint, nobelprize_query)



Querying Nobel Prize Knowledge Graph (Female laureates):
Im here now
Im here now2
Query '
    PREFIX nobel: <http://data.nobelprize.org/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?name_laur WHERE {
    ?laur rdf:type nobel:Laureate .
    ?laur rdfs:label ?name_laur .
    ?laur foaf:gender "female" . }
    ' failed. Attempts: 3
Im here now
Im here now2
Query '
    PREFIX nobel: <http://data.nobelprize.org/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?name_laur WHERE {
    ?laur rdf:type nobel:Laureate .
    ?laur rdfs:label ?name_laur .
    ?laur foaf:gender "female" . }
    ' failed. Attempts: 2
Im here now
Im here now2
Query '
    PREFIX nobel: <http://data.nobelprize.org/terms/>
    PREFIX 

In [25]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "http://data.nobelprize.org/sparql"
)


# gets the first 3 geological ages
# from a Geological Timescale database,
# via a SPARQL endpoint
sparql.setQuery("""
    PREFIX nobel: <http://data.nobelprize.org/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?laur WHERE {
    ?laur rdf:type nobel:Laureate .
     }
    """
)

sparql.setReturnFormat(JSON)


results = sparql.query().convert()
ret = sparql.queryAndConvert()
print(results)

b'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <link rel="stylesheet" type="text/css" href="assets/nobel.css" />\n    <link rel="stylesheet" type="text/css" href="assets/entryscape.css" />\n    <link href="https://unpkg.com/@triply/yasgui/build/yasgui.min.css" rel="stylesheet" type="text/css" />\n    <script src="https://unpkg.com/@triply/yasgui/build/yasgui.min.js"></script>\n\n    <title>Nobelprizes - search</title>\n</head>\n<body>\n<header role="banner">\n    <div>\n        <h1>\n            <a href="https://www.nobelprize.org/">\n                <img src="assets/nobel-logo.svg" alt="logo-title" />\n            </a>\n        </h1>\n    </div>\n    <nav id="main-navigation-js" role="navigation" aria-label="Main menu">\n        <ul role="menu">\n 

**Task 2**

Query the same remote graph for the nobel dataset to find:

1.  List the names of all persons who won the Nobel Prize in Chemistry (hint: the full name of a Nobel Prize laureate can be accessed using the foaf:name property).

2. For all Nobel Prize laureates from the year 2000 [Prize Winners in the year 2000], list their name, date of birth, and date of death if the person has passed away (hint: use an OPTIONAL clause in your query).

3. What is the birth year of persons who won the Nobel Prize in Chemistry? (hint: to obtain the birth year as a query variable you may use a BIND clause with the year function).

4. For every Nobel Prize category, what is the average age of persons who won the Nobel Prize in the category?

5. Who won the Nobel Prize in Chemistry in 1911?




In [24]:
#Task 2
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "http://data.nobelprize.org/sparql"
)



#Query 1

sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix chem:<http://data.nobelprize.org/resource/category/>
select  ?person {
	?s ?p chem:Chemistry.
  ?s rdfs:label ?person
}
    """
)

sparql.setReturnFormat(JSON)


results = sparql.query().convert()
ret = sparql.queryAndConvert()
print("Results for Query 01: \n", results)




#Query 2
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nb-21i0284: <http://data.nobelprize.org/terms/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX category: <http://data.nobelprize.org/resource/category/>
PREFIX laureate: <http://data.nobelprize.org/resource/laureateaward/>

SELECT ?label ?DOB ?DOD
 {
  ?s ?p  <http://data.nobelprize.org/terms/Laureate>.
  ?s <http://data.nobelprize.org/terms/laureateAward> ?awardDetails.
  ?awardDetails <http://data.nobelprize.org/terms/year> 2000 .
  ?s rdfs:label ?label.
  ?s <http://dbpedia.org/property/dateOfBirth> ?DOB.
    optional{ ?s <http://dbpedia.org/property/dateOfDeath> ?DOD.}
}

    """
)

sparql.setReturnFormat(JSON)


results = sparql.query().convert()
ret = sparql.queryAndConvert()
print("Results for Query 02: \n", results)




#Query 3
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nb-21i0284: <http://data.nobelprize.org/terms/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX chem: <http://data.nobelprize.org/resource/category/>
PREFIX laureate: <http://data.nobelprize.org/resource/laureateaward/>

SELECT ?label ?DOB ?year
 {
  ?s ?p  chem:Chemistry.
  ?s <http://data.nobelprize.org/terms/laureate> ?laureate.
  ?laureate rdfs:label ?label.
  ?laureate <http://dbpedia.org/property/dateOfBirth> ?DOB.
    BIND(YEAR(?DOB) AS ?year)
} order by ?year

    """
)

sparql.setReturnFormat(JSON)


results = sparql.query().convert()
ret = sparql.queryAndConvert()
print("Results for Query 03: \n", results)




#Query 4
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nb-21i0284: <http://data.nobelprize.org/terms/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX category: <http://data.nobelprize.org/resource/category/>
PREFIX laureate: <http://data.nobelprize.org/resource/laureateaward/>

SELECT ?catLabel (AVG((?winningYear - ?year)) AS ?averageAgeForEachCategory)
WHERE {
  ?s ?p ?category.
  ?s <http://data.nobelprize.org/terms/year> ?winningYear.
  ?s <http://data.nobelprize.org/terms/laureate> ?laureate.
  ?laureate rdfs:label ?label.
  ?laureate <http://dbpedia.org/property/dateOfBirth> ?DOB.
  BIND(YEAR(?DOB) AS ?year)
  FILTER (?category IN (category:Chemistry, category:Physics, <http://data.nobelprize.org/resource/category/Physiology_or_Medicine>, category:Literature, category:Peace, <http://data.nobelprize.org/resource/category/Economic_Sciences>))
}
GROUP BY ?catLabel

    """
)

sparql.setReturnFormat(JSON)


results = sparql.query().convert()
ret = sparql.queryAndConvert()
print("Results for Query 04: \n", results)




#Query 5
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix nb-21i0284: <http://data.nobelprize.org/terms/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix chem:<http://data.nobelprize.org/resource/category/>
prefix laureate:<http://data.nobelprize.org/resource/laureateaward/>
select  ?label {
	?s ?p chem:Chemistry.
  ?s rdfs:label ?label.
    ?s <http://data.nobelprize.org/terms/year> 1911
}

    """
)

sparql.setReturnFormat(JSON)


results = sparql.query().convert()
ret = sparql.queryAndConvert()
print("Results for Query 05: \n", results)

/usr/local/lib/python3.10/dist-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html' returning raw response...
  warnings.warn(


Results for Query 01: 
 b'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <link rel="stylesheet" type="text/css" href="assets/nobel.css" />\n    <link rel="stylesheet" type="text/css" href="assets/entryscape.css" />\n    <link href="https://unpkg.com/@triply/yasgui/build/yasgui.min.css" rel="stylesheet" type="text/css" />\n    <script src="https://unpkg.com/@triply/yasgui/build/yasgui.min.js"></script>\n\n    <title>Nobelprizes - search</title>\n</head>\n<body>\n<header role="banner">\n    <div>\n        <h1>\n            <a href="https://www.nobelprize.org/">\n                <img src="assets/nobel-logo.svg" alt="logo-title" />\n            </a>\n        </h1>\n    </div>\n    <nav id="main-navigation-js" role="navigation" aria-label="Main menu">\n   

**DBPedia Knowledge Graph**

These are some example queries about “Emma Watson”. These queries are based on the DBpedia Knowledge Graph, a RDF version of Wikipedia (https://dbpedia.org/), and can be tested via its SPARQL Endpoint (http://dbpedia.org/sparql).

SPARQL Examples
Example (i)
Films starring people called “Emma Watson”
PREFIX foaf: http://xmlns.com/foaf/0.1/

PREFIX dbo: http://dbpedia.org/ontology/

SELECT ?m

WHERE

{

    ?jd foaf:name "Emma Watson"@en .
    ?m dbo:starring ?jd .
}

Example (ii)

Names of people who co-starred with “Emma Watson”
SELECT DISTINCT ?costar

WHERE

{

    ?jd foaf:name "Emma Watson"@en .
    ?m dbo:starring ?jd .
    ?m dbo:starring ?other .
    ?other foaf:name ?costar .

}



**Task 3**
Try these queries using the DBPedia Endpoint programmatically

In [23]:
#Your code for Task 3

from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pprint import pprint
import time



def queryRemoteGraph(endpoint_url, query, attempts=3):


    try:

        sparql_web = SPARQLWrapper(endpoint_url)
        # Default is XML:
        # https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.Wrapper.html
        sparql_web.setReturnFormat(JSON)
        # sparql_web.setReturnFormat(XML)

        sparql_web.setQuery(query)
        print("Im here now")
        results = sparql_web.query().convert()
        ret = sparql_web.queryAndConvert()
        print("Im here now2")
        for r in ret["results"]["bindings"]:
         print(r)
         print("Im here now3")

      #  Prints JSON file
        print("Im here now to check")
        print("RAW RESULTS IN JSON FORMAT:")
        print(results)


        print("\nRESULTS:")
        print("Im here now4")
        for result in results["results"]["bindings"]:

            #Prints individual results
            print(result["name_laur"]["value"])
            print("Im here now5")




    except:

        print("Query '%s' failed. Attempts: %s" % (query, str(attempts)))
        #time.sleep(60) #to avoid limit of calls, sleep 60s
        attempts-=1
        if attempts>0:
            return queryRemoteGraph(endpoint_url, query, attempts)
        else:
            return None



#See more examples here: https://www.nobelprize.org/about/linked-data-examples/
# Query 2
dbpedia_endpoint = "http://dbpedia.org/sparql"
dbpedia_query = """
    select ?m
    where
    {
      ?jd foaf:name "Emma Watson"@en .
      ?m dbo:starring ?jd .
    }
    """



print("\nQuerying Nobel Prize Knowledge Graph (Female laureates):")
queryRemoteGraph(dbpedia_endpoint, dbpedia_query)



# Query 2
dbpedia_query = """
select ?costar where
{
?jd foaf:name "Emma Watson"@en .
?m dbo:starring ?jd .
?m dbo:starring ?other .
?other foaf:name ?costar .}"""

queryRemoteGraph(dbpedia_endpoint, dbpedia_query)


Querying Nobel Prize Knowledge Graph (Female laureates):
Im here now
Im here now2
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Production_of_Harry_Potter_and_the_Deathly_Hallows'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Beauty_and_the_Beast_(2017_film)'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Little_Women_(2019_film)'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Regression_(film)'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Colonia_(film)'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/My_Week_with_Marilyn'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Perks_of_Being_a_Wallflower_(film)'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Tale_of_Despereaux_(film)'}}
Im here now3
{'m': {'type': 'uri', 'value': 'http://dbpedia.org/resource/This_Is_the_End'}

---

**Note:** I have tried my best to provide accurate results in this notebook. However, these results may not be entirely accurate, and contributions or corrections are encouraged. Thank you!
